# Image Classification Model - Serving Function

This notebook demonstrates how to deploy a Tensorflow model using MLRun & Nuclio.

**In this notebook you will:**
* Write a Tensorflow-Model class to load and predict on the incoming data
* Deploy the model as a serverless function
* Invoke the serving endpoint with data as:
  * URLs to images hosted on S3
  * Direct image send
  
**Steps:**  
* [Define Nuclio function](#Define-Nuclio-function)  
  * [Install dependencies and set config](#Install-dependencies-and-set-config)  
  * [Model serving class](#Model-Serving-Class)  
* [Deploy the serving function to the cluster](#Deploy-the-serving-function-to-the-cluster)  
* [Define test parameters](#Define-test-parameters)
* [Test the deployed function on the cluster](#Test-the-deployed-function-on-the-cluster)

## Define Nuclio Function

To use the magic commands for deploying this jupyter notebook as a nuclio function we must first import nuclio  
Since we do not want to import nuclio in the actual function, the comment annotation `nuclio: ignore` is used. This marks the cell for nuclio, telling it to ignore the cell's values when building the function.

In [1]:
# nuclio: ignore
import nuclio

### Install dependencies and set config
> Note: Since tensorflow is being pulled from the baseimage it is not directly installed as a build command.
If it is not installed on your system please uninstall and install using the line: `pip install tensorflow`

In [2]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=TF2Model

# tensorflow 2 use the default serving image (or the mlrun/ml-models for a faster build)

%nuclio config spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


Since we are using packages which are not surely installed on our baseimage, or want to verify that a specific version of the package will be installed we use the `%nuclio cmd` annotation.  
>`%nuclio cmd` works both locally and during deployment by default, but can be set with `-c` flag to only run the commands while deploying or `-l` to set the variable for the local environment only.

In [3]:
%%nuclio cmd -c
pip install tensorflow>=2.1
pip install requests pillow

## Function Code

In [4]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [5]:
import json
import numpy as np
import requests
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img
from os import environ, path
from PIL import Image
from io import BytesIO
from urllib.request import urlopen
import mlrun

### Model Serving Class

We define the `TFModel` class which we will use to define data handling and prediction of our model.  

The class should consist of:
* `__init__(name, model_dir)` - Setup the internal parameters
* `load(self)` - How to load the model and broadcast it's ready for prediction
* `preprocess(self, body)` - How to handle the incoming event, forming the request to an `{'instances': [<samples>]}` dictionary as requested by the protocol
* `predict(self, data)` - Receives and `{'instances': [<samples>]}` and returns the model's prediction as a list
* `postprocess(self, data)` - Does any additional processing needed on the predictions.

In [6]:
class TFModel(mlrun.runtimes.MLModelServer):
    def __init__(self, name: str, model_dir: str):
        super().__init__(name, model_dir)

        self.IMAGE_WIDTH = int(environ.get('IMAGE_WIDTH', '128'))
        self.IMAGE_HEIGHT = int(environ.get('IMAGE_HEIGHT', '128'))
        
        try:
            with open(environ['classes_map'], 'r') as f:
                self.classes = json.load(f)
        except:
            self.classes = None
        
    def load(self):
        model_file, extra_data = self.get_model('.h5')
        self.model = load_model(model_file)
        
    def preprocess(self, body):
        try:
            output = {'instances': []}
            instances = body.get('instances', [])
            for byte_image in instances:
                img = Image.open(byte_image)
                img = img.resize((self.IMAGE_WIDTH, self.IMAGE_HEIGHT))

                # Load image
                x = image.img_to_array(img)
                x = np.expand_dims(x, axis=0)
                output['instances'].append(x)
            
            # Format instances list
            output['instances'] = [np.vstack(output['instances'])]
            return output
        except:
            raise Exception(f'received: {body}')
            

    def predict(self, data):
        images = data.get('instances', [])

        # Predict
        predicted_probability = self.model.predict(images)

        # return prediction
        return predicted_probability
        
    def postprocess(self, predicted_probability):
        predicted_probabilities = np.max(predicted_probability.tolist()[0])
        return {'Mask probability': predicted_probabilities}

To let our nuclio builder know that our function code ends at this point we will use the comment annotation `nuclio: end-code`.  

Any new cell from now on will be treated as if a `nuclio: ignore` comment was set, and will not be added to the funcion.

In [7]:
# nuclio: end-code